In [23]:

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

class DozeModel:
  def __init__(self, X, y, n=None, tolerance=1e-10, max_iter=50, use_pca=False, use_scaler=False):
    # تبدیل DataFrame به numpy array در صورت نیاز
    if isinstance(X, pd.DataFrame):
      X = X.values
    if isinstance(y, pd.Series):
      y = y.values

    self.X = X
    self.y = y
    self.n = np.ones(len(y)) if n is None else n  # اگر n داده نشود، مقدار پیشفرض 1 است
    self.tolerance = tolerance
    self.max_iter = max_iter
    self.use_pca = use_pca
    self.use_scaler = use_scaler

    # اعمال StandardScaler در صورت نیاز
    if self.use_scaler:
      self.scaler = StandardScaler()
      self.X = self.scaler.fit_transform(self.X)

    # اعمال PCA در صورت نیاز
    if self.use_pca:
      self.pca = PCA()
      self.X = self.pca.fit_transform(self.X)

    self.b = np.zeros((self.X.shape[1], max_iter))
    self.z = np.zeros((len(y), max_iter))
    self.yhat = np.zeros((len(y), max_iter))
    self.syhat = np.zeros((self.X.shape[1], max_iter))
    self.kai2 = np.zeros(max_iter)
    self.e = np.zeros((self.X.shape[1], max_iter))
    self.sy = np.zeros(self.X.shape[1])
    self.p = np.zeros((len(y), max_iter))
    self.p[:, 0] = (y + 0.5) / (self.n + 0.5)
    self.yhat[:, 0] = y

  def fit(self):
    for i in range(self.max_iter - 1):
      self.p[:, i] = np.clip(self.p[:, i], self.tolerance, 1 - self.tolerance)
      self.z[:, i] = np.log(self.p[:, i] / (1 - self.p[:, i])) + (self.y - self.n * self.p[:, i]) / (self.n * self.p[:, i] * (1 - self.p[:, i]))
      W = np.diag(self.n * self.p[:, i] * (1 - self.p[:, i]))
      self.b[:, i + 1] = np.linalg.solve(self.X.T @ W @ self.X, self.X.T @ W @ self.z[:, i])
      xb = np.clip(self.X @ self.b[:, i + 1], -500, 500)
      self.p[:, i + 1] = np.exp(xb) / (1 + np.exp(xb))
      self.yhat[:, i + 1] = self.n * self.p[:, i + 1]
      self.syhat[:, i + 1] = self.yhat[:, i + 1] @ self.X
      self.sy = self.y @ self.X
      self.e[:, i + 1] = self.syhat[:, i + 1] - self.sy
      self.kai2[i + 1] = np.sum(((self.y - self.yhat[:, i + 1]) ** 2) / self.yhat[:, i + 1])
    return {
    'coefficients': self.b,
    'predicted_probabilities': self.p,
    'yhat': self.yhat,
    'syhat': self.syhat,
    'errors': self.e,
    'kai2': self.kai2
    }

# استفاده از کلاس
X = pd.DataFrame([[1, 2], [2, 3], [3, 4], [4, 5]])
y = pd.Series([0, 1, 0, 1])
model = DozeModel(X, y, tolerance=1e-8, max_iter=100, use_pca=True, use_scaler=True)
results = model.fit()
print(results['coefficients'][:,-1])


[-1.34766547e+07 -1.04045934e+23]


In [ ]:
pip install ucimlrepo

In [25]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# تبدیل y به آرایه numpy و تبدیل مقادیر متنی به عددی
y = y.replace({'M': 1, 'B': 0}).values.ravel()


<ipython-input-25-3e07b9e95726>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'M': 1, 'B': 0}).values.ravel()


In [26]:
model = DozeModel(X, y, tolerance=1e-8, max_iter=100, use_pca=True, use_scaler=True)
results = model.fit()
print(results['coefficients'][:,-1])

[ 2.22930580e+01 -1.55739057e+01 -5.08491945e+00  1.98367102e+00
  1.29158855e+01 -4.92183833e+00  8.19428025e+00  8.59205094e+00
  2.60861771e+01 -2.12423521e+01 -8.53843755e+00  9.10364248e+00
  1.59204052e-01 -8.85217254e+00 -1.62798455e+01  7.89848668e+00
  5.03571329e+01  5.72756047e+00  3.87820388e+01 -4.22166684e+01
  2.84767027e+01  7.23647391e+00 -2.41982431e+01  3.20107097e+01
 -1.35348485e+01  2.91396040e+01 -1.51210246e+01  9.76137411e+01
  2.05644379e+02 -3.31190353e+02]
